# Calculation sheet for vapour cells

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import *

# Coherence time
<br>

$$\Gamma =\frac{1}{T_2} = \frac{1}{T_1} + \frac{1}{q_{SE}} R_{SE} + \Gamma_{gr}$$
with :
- $R_{SE}$ the rate of spin-exchange collisions
- $R_{gr}$ to broadening due to magnetic field gradient across the cell
- $T_1$ as follows :
$$\frac{1}{T1} = \frac{1}{q} (R_{SD} + R_{OP} + R_{pr}) + \Gamma_{wall}$$
The nuclear slowing down factor $q = 2I+1$ in the case of high polarization ($P\approx 1$) ; $R_{SD}$ is the spins-destruction collision rate ; $R_{OP}$ is the optical pumping rate ; $R_{pr}$ is the absorption rate of photons for a probe beam, irrelevant in our case (probe beam being the same as the excitation beam) ; $\Gamma_{wall}$ is the depolarization rate due to collisions with the walls of the vapor cell, destroying both electron and nuclear spins. The spins destruction rate decomposes as follows :

$$R_{SD} = \Gamma^{self}_{SD} + \Gamma^{B}_{SD} + \Gamma^{Q}_{SD}$$

With the first term being the collisions between alkali atoms, the second collisions with buffer gas atoms, the third collisions with quenching gas molecules


The general collision rate is given by :
$$\Gamma_{collisions} = n\sigma \bar{v}$$
with :
$$\bar{v} = \sqrt{\frac{8k_BT}{\pi M}}$$
$$\frac{1}{M} = \frac{1}{m} + \frac{1}{m\prime}$$
with m the masses of the alkali atoms and m' the mass of the perturber ; n the the density of the other gas species ; $\sigma$ the effective collisional cross section.
<br>

### Spin exchange relaxation
At high density, the spin exchange relaxation is the dominant cause of relaxation. We thus want to reduce it, either by reducing the density (acting on $\Gamma_{SE} = n_{alkali}\sigma_{alkali}\bar{v}$) or by having aa shielding and an extremely low magnetic field, using the so called SERF regime. Here, we are aiming for an unshielded sensor, so the solution will be to reduce the density.
In our case, we can assume a 'high' magnetic field (no shielding, earth magnetic field), yielding a Larmor frequency much higher than the spin exchange rate $\mid \omega_{\pm} \mid \gg \Gamma_{SE}$, we thus have :
$$\frac{1}{q_{SE}} = \frac{2I(2I-1)}{3(2I+1)^2}$$
to be plugged back in the equation of the T2.
<br>

### Spin destruction relaxation
The spin destruction relaxation is the second most important mechanism for spin relaxation that do not preserve the total spin of the alkali ensemble. These collisions occur between alkali atoms, or between alkali atoms and buffer and quenching gases, and is thus given by the standard rate for collisions.

### Wall collisions
The alkali atoms going close to the walls, usually containing ions, will experience an increasing electric field as it gets closer, yielding a magnetic field, which depolarizes the atom. This effect can be mitigated by adding a buffer gas to reduce the probability of collisions with a wall using inert gases, or coatings on the walls to reduce drastically this gradient of electric field experienced by the atom. Wall collisions are completely depolarizing, dominating other spin relaxation mechanisms if not suppressed.
<br>
Most effective coatings usually melt at 60-80°C, except for OTS up to 170°C (see Scott F. Seltzer PhD Thesis). The point of the coating is to allow atoms to bounces thousands of times before depolarizing, reducing the rate by the same factor.
<br>
Regarding buffer gases, for a spherical cavity with high pressure inert buffer gaz, the rate is the following :
$$ \Gamma_{wall}=\frac{1}{T_{wall}} = D \left( \frac{\pi}{R} \right)^2 $$
with R the radius, pi the constant and $D = \tfrac{1}{3}\lambda\bar{v}$ and $\lambda$ the mean free path length.

### Magnetic field gradients
A magnetic field gradient creates a difference in the Larmor frequency of the alkali atoms population, hence a broadening of the resonance. In a coated cell, the alkali atoms travel freely across the cell, sampling the entire volume and average the magnetic field. This somehow removes the gradient effect (motional narrowing).
<br>
At high buffer pressure, alkali atoms are more limited in space and experience one single Larmor Frequency (slow diffusion). In this regime, the gradient broadening is given by the spread of precession frequencies through the cell as follows :
$$\Gamma_{gr} \approx \gamma \nabla B$$

<br><br>
In our case, we'll simulate with what the CSEM does at first, and then try to adjust. We assume the pressure flexible, and that we use the following vapor :
$$RbN_3 \rightarrow Rb + \tfrac{3}{2}N_2$$
**We must find the max values of each decoherence factor in order to achieve a $\Gamma$ close to 1 - 3000 Hz.**

In [23]:
## functions for the different spin relaxation mechanisms

def v_mean(T, M):
    v = np.sqrt((8*k*T)/(pi*M))
    return v

def collision_mechanism(n, sigma, v_mean):
    R_col = n*sigma*v_mean
    return R_col

def inverse_reduced_mass(m1,m2):
    inverse_reduced_mass = 1/m1 + 1/m2
    return inverse_reduced_mass

def nuclear_slowing_down_factor(I):
    q = 2*I+1
    return q

def nuclear_slowing_down_factor_spin_exchange(I):
    q_SE = (3*(2*I+1)**2)/(2*I*(2*I-1))
    return q_SE

def number_density_gas(P, T):
    n = (P)/(k*T)
    return n

def number_density(T):
    n = 1/T*10**(21.866+4.312-4040/T)
    return n

def matter_quantity(P, V, T):
    n = (P*V)/(R*T)
    return n

 ### constants


In [24]:
## Numbers we want
V = 3e-3*3e-3*1.9e-3 #m^3
P = 3e4 #Pa
T = 273.15+60

R = 3e-3
L = 1.9e-3


## useful values, the 1e-4 factor is to turn from cm^2 to m^2
## Values taken from [1]
sigma_SE_self = 1.9e-14 * 1e-4 #m^2
sigma_SD_self = 1.6e-17 * 1e-4
sigma_SD_N2 = 1e-22 * 1e-4
sigma_Q_n2 = 5.8e-15 * 1e-4
D_N2 = 0.19 * 1e-4
m_Rb = 1.44e-25 #kg, for 87Rb
m_N2 = 4.65e-26
I_Rb = 3/2
gamma_Rb = 7e9
r_e = 2.82e-15 #m
f_D1 = 0.332
## Values taken from [2]
sigma_D1_absorption = 1.082e-13 #m^2 from Table 7, Rubidium 87 D Line Data, Daniel A. Steck
Gamma_natural = 36.10e6 #Hz
## Value from [4]
L0 = 2.65e25 #m^-3, Loschmidt constant
D_0 = 23e-5 #m^2.s^-1
T_0 = 273.15+100
P_0 = 1.01325e5 #Pa

### Spin Exhange Relaxation Rate
This relaxation rate will be given by $\Gamma_{SE} = \frac{1}{q_{SE}}  R_{SE}$. <br>
In the case of Rb atoms, $I = \tfrac{3}{2}$. <br>
The interaction takes place between alkali atoms only, no need for reduced mass.

In [25]:
q_SE_Rb = nuclear_slowing_down_factor_spin_exchange(I_Rb)

v_mean_Rb = v_mean(T, m_Rb)

#n_Rb = (2/5)*number_density(P, T)
n_Rb = (2/5)*number_density(T)
R_SE = collision_mechanism(n_Rb, sigma_SE_self, v_mean_Rb) #Hz

Gamma_SE = (R_SE*(1/q_SE_Rb))

In [26]:
Gamma_SE

np.float64(9.15302146016331e-06)

### Spin Destruction Relaxation Rate
This relaxation is given only by the collision between alkali atoms and buffer gas, quenching gas, or between alkali atoms only. <br>

Regarding the collision with the buffer gas, the following expression is given in [4] :

$$R_{Rb-N^2} = L_0  \bar{v}_{rel, Rb-N_2}  \sigma_{N_2}   \frac{P}{P_0} $$

In [27]:
reduced_mass = 1/inverse_reduced_mass(m_N2, m_Rb)
v_mean_N2 = v_mean(T, reduced_mass)
n_N2 = (3/5)*number_density_gas(P, T)

R_buffer_gas = L0*v_mean_N2*sigma_SD_N2*P/P_0

R_SD_self = collision_mechanism(n_Rb, sigma_SD_self, v_mean_Rb)
R_SD_N2 = L0*v_mean_N2*sigma_SD_N2*P/P_0
R_SD_Q = 0

R_SD = R_SD_self + R_buffer_gas + R_SD_Q

In [28]:
R_SD

np.float64(45.292040984935355)

### Wall Collisions

Regarding buffer gases, for a spherical cavity with high pressure inert buffer gaz, the rate is the following :
$$\Gamma_{wall} = \frac{1}{T_{wall}} = D \left( \frac{\pi}{R} \right)^2$$
with R the radius, pi the constant and $D = \tfrac{1}{3}\lambda\bar{v}$ and $\lambda$ the mean free path length. For the sake of us, the $D_0^{N_2} = 0.19 \,cm^2.s^{-1}$ is given in Developments in Alkali-Metal Atomic Magnetometer, Scott F. Seltzer, table A.2.
<br>
Here, the cell isn't spherical, but a cylinder where the beam will cross through its height. The polarization function is approximated as its fundamental diffusion mode :
$$ P(r,t) = \frac{sin(kr)}{kr}e^{- \tfrac{t}{T_{wall}}}$$
with the boundary condition $P(h)=0$ yields $k= \frac{\pi}{h}$

For lower pressure, or maybe in general, the cell wall relaxation rate is given by [4] :
$$\Gamma_{wall} = \left[ \left(\frac{2.405}{R}\right)^2 + \left( \frac{\pi}{L} \right)^2 \right]D_0 \frac{P_0}{P} \left(\frac{T}{T_0} \right)^{3/2} $$
with:
- R the cell radius
- L the cell length
- $D_0$ the Rb atoms diffusion constant; for $T_0 = 100°C ;\; P_0=760\, torr$, $D_0 = 0.23 \,cm^2.s^{-1}$
- T the cell temperature
- P the cell pressure.


In [29]:
Gamma_wall_general = ((2.405/R)**2 + (np.pi/L)**2)*D_0*P_0/P*(T/T_0)**(3/2)
Gamma_wall_general

2212.796459953226

In [30]:
height_cell = 1.9e-3 #m^3

Gamma_wall_high_pressure = D_N2*(pi/height_cell)**2

In [31]:
Gamma_wall_high_pressure

51.94528632152294

### Magnetic Field Gradient

We, for now, assume this one as irrelevant since we'll have a very homogeneous field across the cell for the benchtop experiment. If more data about the gradient strength of the heart magnetic field is acquired, just plug it in!
The expression is the following :
$$\Gamma_{gr} \approx \gamma \nabla B$$

In [32]:
grad_B = 192e-9

Gamma_gradient = gamma_Rb*grad_B

In [33]:
Gamma_gradient

1344.0

### Optical Absorption Rate

The optical absorption rate can be generally described as follows :
$$R_{abs} = \sum_{res} \sigma(\nu)\phi(\nu)$$
with $\sigma(\nu)$ the cross section per resonant frequency and $\phi(\nu)$ the optial flux per resonant frequency. In our case, we're using a laser at 795nm, which collapses the sum.
<br>
The optical flux is easily computed as the amount of photon per area per time $\phi(\nu) = \frac{I}{E_{photon}} = \frac{\tfrac{P_{laser}}{A}}{E_{photon}}$, the cross section will be taken from Rubidium 87 D Line Data, Daniel A. Steck, table 7. <br>
Nevertheless, the optical power might be too large and we must take into account the saturation of the Rb atoms. For that, we use the natural spontaneous decay time also given in Rubidium 87 D Line Data, Daniel A. Steck, table 4, as $\Gamma_{natural} = 36.10 \, MHz$.
<br>
We will also need the saturation intensity, given as $I_{sat} = 44.84 \, W.m^{-2}$. <br>
With saturation effects, we have :
We have $$\Gamma_{abs} = \sigma_{abs}\phi$$
in the case of saturation, we have
$$\Gamma_{abs} = \Gamma_{natural}$$
$$\sigma_{abs}\phi = \Gamma_{natural}$$
since $\phi = \frac{I}{h\nu}$, we have:
$$\frac{I_{sat}}{h\nu}\sigma_{abs} = \Gamma_{natural}$$
Hence
$$I_{sat} = \frac{h\nu\Gamma_{natural}}{\sigma_{abs}}$$

That way, computing $R_{abs}$ with saturation effect leads to :
$$R_{abs} = R_{sat} \cdot  \frac{\tfrac{I}{I_{sat}}}{1+\tfrac{I}{I_{sat}}}$$
As in $R_{abs}$ will be a portion of the maximum $R_{sat}$ if $I\ll I_{max}$ and equal to $R_{sat}$ if $I \gg I_{sat}$.
$I_{sat}$ represents here the intensity at which the system moves from linear absorption to saturated absorption.

In [34]:
## Laser paremeters ##
laser_wavelength = 795e-9
P_laser = 1e-6
beam_diameter = 1e-3 #m
I = P_laser/(pi*(beam_diameter/2)**2)
photon_flux = I/((h*c)/laser_wavelength)

I_sat = 44.84

R_abs = Gamma_natural * (I)/(I+I_sat)

#R_abs = sigma_D1_absorption*photon_flux #Cannot work, units issue, find the value of the cross section, not the integrated one over the frequency

R_OP = R_abs
print('the absorption rate is : {:.3E} Hz'.format(R_abs))

the absorption rate is : 9.968E+05 Hz


In [35]:
R_op_no_sat = sigma_D1_absorption*photon_flux
R_op_no_sat

551350.5037372019

In [36]:
R_abs/Gamma_natural

0.02761114936416413

### Compute the $\Gamma$ value

We now should have everything to compute the value for our given parameters.
$$\Gamma = \frac{1}{T_2} = \frac{1}{q} (R_{SD}+R_{OP}+R_{pr}) + \frac{1}{q_{SE}}R_{SE} + \Gamma_{gr} + \Gamma_{wall}$$
with $R_{pr}$ and $\Gamma_{gr}$ irrelevant for us.

In [37]:
q = nuclear_slowing_down_factor(3/2)

Gamma = 1/q * (R_SD + R_OP) + Gamma_SE + Gamma_gradient + Gamma_wall_general

In [38]:
Gamma

np.float64(252758.74249093374)

In [39]:
Gamma_no_ROP = 1/q * (R_SD) + Gamma_SE + Gamma_gradient + Gamma_wall_general

In [40]:
Gamma_no_ROP

np.float64(3568.1194793524814)

# RF power calculations

The $B_{rf}(t)$ field to excite the zeeman sublevels will be written as $B_{rf}(t) = B_{rf} \cos(\omega_{rf} t)$. The amplitude of the magnetic field will be given by :
$$ B_{rf} = \frac{\mu_0 N I_{rf}}{2r} $$
with :
- $I_{rf}$ the current intensity
- $N$ the number of turns
- $r$ the radius of the loop

Assuming that we want $B_{rf}$ of about 1 mT:

$$I_{rf} = \frac{2B_{rf} r}{\mu_0 N}$$

The RF power is :
$$ P_{rf} = RI_{rf}^2$$

And the radiated power from magnetic dipole is (see markdown below) :
$$P_{rad} = \frac{\mu_0^2 \omega^4 m^2}{12 \pi c^3}$$
with :
- m : magnetic moment $m = N I A$

In our case, the $B_{rf}$ will need to match the number of Rb atom we have.

A small coil (i.e., one whose dimensions are much smaller than the wavelength of the radiation) behaves like an oscillating magnetic dipole. The instantaneous power radiated by a magnetic dipole with moment $m(t)$ is given by

$$P(t) = \frac{\mu_0}{6\pi c^3} \, \ddot{m}(t)^2,$$

where:
- $\mu_0$ is the permeability of free space,
- $c$ is the speed of light,
- $\ddot{m}(t)$ is the second derivative of the magnetic dipole moment with respect to time.

Assume the magnetic dipole moment oscillates as

$$m(t) = m_0 \cos(\omega t),$$

where $m_0$ is the amplitude and $\omega$ is the angular frequency. Its second derivative is

$$\ddot{m}(t) = -\omega^2 m_0 \cos(\omega t).$$

Thus, we have

$$\ddot{m}(t)^2 = \omega^4 m_0^2 \cos^2(\omega t).$$

Since the time average of \(\cos^2(\omega t)\) over one cycle is

$$\langle \cos^2(\omega t) \rangle = \frac{1}{2},$$

the time-averaged radiated power becomes

$$\langle P \rangle = \frac{\mu_0}{6\pi c^3}\,\omega^4 m_0^2 \cdot \frac{1}{2} = \frac{\mu_0\,\omega^4\,m_0^2}{12\pi c^3}.$$

If the coil has $N$ turns, carries an oscillatory current with amplitude \( I_0 \), and has a cross-sectional area \( A \), then its magnetic dipole moment is

$$m_0 = N I_0 A.$$

Substituting this into the power expression gives
$$\langle P \rangle = \frac{\mu_0\,\omega^4\,(N I_0 A)^2}{12\pi c^3}.$$

For example, for a single-turn coil (\(N = 1\)) with radius \(a\) (so that \(A = \pi a^2\)), the radiated power is
$$\langle P \rangle = \frac{\mu_0\,\omega^4\,I_0^2\,(\pi a^2)^2}{12\pi c^3}= \frac{\mu_0\,\omega^4\,\pi\,a^4\,I_0^2}{12 c^3}.$$

In [41]:
N = 10
B_rf = 1e-3
r = 3e-3
R= 2 #resistance of our copper wire, overestimated here if you ask me
omega = 200e3

I_rf = B_rf*2*r/(mu_0*N)

P_rf = R*I_rf**2

P_rad = mu_0**2 * omega**4 * (N*I_rf*pi*r**2)**2 / (12*np.pi*c**3)

print('RF current : {0:.2e}\nRF Power : {1:.2e}\nPower radiated : {2:.2e}'.format(I_rf, P_rf, P_rad))

RF current : 4.77e-01
RF Power : 4.56e-01
Power radiated : 4.53e-26


### Assumptions & remarks

The part about the optical pumping is flawed, must remake it dynamically considering the evolution of the populations in the system (2-level system, typically) <br>
The $\Gamma_{wall}$ is calculated under the assumption of high buffer gas pressure, which isn't the case since we want a cell with low pressure. To be adjusted, also the formula seems weird, a longer mean free path induces a shorter spin relaxation time, doesn't make sense to me. <br>

# Sources :

[1] : 'Developments in Alkali-Metal Atomic Magnetometers', Scott F. Seltzer. <br>
[2] : 'Rubidium 87 D Line Data', Daniel A. Steck
<br>
[3] : 'A moveable unshielded magnetocardiography system', W Xiao, C Sun, L Shen et al.<br>
[4] : 'Fabrication and characterization of MEMS alkali vapor cells used in chip-scale atomic clocks and other atomic devices', S. Karlen.
